In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
%matplotlib inline

In [2]:
train = pd.read_csv('/Users/wenjiazhai/Desktop/员工离职预测训练赛/pfm_train.csv')
test = pd.read_csv('/Users/wenjiazhai/Desktop/员工离职预测训练赛/pfm_test.csv')

In [3]:
train.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,37,0,Travel_Rarely,Research & Development,1,4,Life Sciences,77,1,Male,2,2,Manufacturing Director,3,Divorced,5993,1,Y,No,18,3,3,80,1,7,2,4,7,5,0,7
1,54,0,Travel_Frequently,Research & Development,1,4,Life Sciences,1245,4,Female,3,3,Manufacturing Director,3,Divorced,10502,7,Y,No,17,3,1,80,1,33,2,1,5,4,1,4
2,34,1,Travel_Frequently,Research & Development,7,3,Life Sciences,147,1,Male,1,2,Laboratory Technician,3,Single,6074,1,Y,Yes,24,4,4,80,0,9,3,3,9,7,0,6
3,39,0,Travel_Rarely,Research & Development,1,1,Life Sciences,1026,4,Female,2,4,Manufacturing Director,4,Married,12742,1,Y,No,16,3,3,80,1,21,3,3,21,6,11,8
4,28,1,Travel_Frequently,Research & Development,1,3,Medical,1111,1,Male,2,1,Laboratory Technician,2,Divorced,2596,1,Y,No,15,3,1,80,2,1,2,3,1,0,0,0


In [4]:
print(train.describe())
print(train.info())

               Age    Attrition  DistanceFromHome    Education  \
count  1100.000000  1100.000000       1100.000000  1100.000000   
mean     36.999091     0.161818          9.427273     2.922727   
std       9.037230     0.368451          8.196694     1.022242   
min      18.000000     0.000000          1.000000     1.000000   
25%      30.000000     0.000000          2.000000     2.000000   
50%      36.000000     0.000000          7.000000     3.000000   
75%      43.000000     0.000000         15.000000     4.000000   
max      60.000000     1.000000         29.000000     5.000000   

       EmployeeNumber  EnvironmentSatisfaction  JobInvolvement     JobLevel  \
count     1100.000000              1100.000000     1100.000000  1100.000000   
mean      1028.157273                 2.725455        2.730909     2.054545   
std        598.915204                 1.098053        0.706366     1.107805   
min          1.000000                 1.000000        1.000000     1.000000   
25%       

In [5]:
label = train.Attrition
print(train.Attrition.value_counts())
test['Attrition']=-1
data = train.append(test).reset_index(drop=True)

0    922
1    178
Name: Attrition, dtype: int64


/Users/wenjiazhai/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [6]:
data.drop(['Over18', 'StandardHours', 'EmployeeNumber'], axis=1, inplace=True)

In [7]:
cat_col = [i for i in data.select_dtypes(object).columns if i not in ['Attrition']]
print(cat_col)

['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime']


In [8]:
from itertools import combinations
from tqdm import tqdm_notebook
cat_cross=[]
for col in tqdm_notebook(combinations(cat_col, 2)):   
    data[str(col[0])+'_'+str(col[1])] = data[col[0]].map(str) + '_' + data[col[1]].map(str)
    cat_cross.append(str(col[0])+'_'+str(col[1]))
data['comprehensive_satisfaction'] = data['JobSatisfaction'].map(str) + '_' + data['EnvironmentSatisfaction'].map(str)

In [9]:
train['MonthlyIncome'] = pd.cut(train['MonthlyIncome'], bins=10)

In [10]:
new_cat_col = [i for i in data.select_dtypes(object).columns if i not in ['Attrition']]

In [11]:
def encode_onehot(df,column_name):
    feature_df=pd.get_dummies(df[column_name], prefix=column_name)
    all = pd.concat([df.drop([column_name], axis=1),feature_df], axis=1)
    return all

In [12]:
for i in new_cat_col:
    data = encode_onehot(data,i)

In [13]:
data.head()

,Age,Attrition,DistanceFromHome,Education,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,Gender_Female,Gender_Male,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes,BusinessTravel_Department_Non-Travel_Human Resources,BusinessTravel_Department_Non-Travel_Research & Development,BusinessTravel_Department_Non-Travel_Sales,BusinessTravel_Department_Travel_Frequently_Human Resources,BusinessTravel_Department_Travel_Frequently_Research & Development,BusinessTravel_Department_Travel_Frequently_Sales,BusinessTravel_Department_Travel_Rarely_Human Resources,BusinessTravel_Department_Travel_Rarely_Research & Development,BusinessTravel_Department_Travel_Rarely_Sales,BusinessTravel_EducationField_Non-Travel_Human Resources,BusinessTravel_EducationField_Non-Travel_Life Sciences,BusinessTravel_EducationField_Non-Travel_Marketing,BusinessTravel_EducationField_Non-Travel_Medical,BusinessTravel_EducationField_Non-Travel_Other,BusinessTravel_EducationField_Non-Travel_Technical Degree,BusinessTravel_EducationField_Travel_Frequently_Human Resources,BusinessTravel_EducationField_Travel_Frequently_Life Sciences,BusinessTravel_EducationField_Travel_Frequently_Marketing,BusinessTravel_EducationField_Travel_Frequently_Medical,BusinessTravel_EducationField_Travel_Frequently_Other,BusinessTravel_EducationField_Travel_Frequently_Technical Degree,BusinessTravel_EducationField_Travel_Rarely_Human Resources,BusinessTravel_EducationField_Travel_Rarely_Life Sciences,BusinessTravel_EducationField_Travel_Rarely_Marketing,BusinessTravel_EducationField_Travel_Rarely_Medical,BusinessTravel_EducationField_Travel_Rarely_Other,BusinessTravel_EducationField_Travel_Rarely_Technical Degree,BusinessTravel_Gender_Non-Travel_Female,BusinessTravel_Gender_Non-Travel_Male,BusinessTravel_Gender_Travel_Frequently_Female,BusinessTravel_Gender_Travel_Frequently_Male,BusinessTravel_Gender_Travel_Rarely_Female,BusinessTravel_Gender_Travel_Rarely_Male,BusinessTravel_JobRole_Non-Travel_Healthcare Representative,BusinessTravel_JobRole_Non-Travel_Human Resources,BusinessTravel_JobRole_Non-Travel_Laboratory Technician,BusinessTravel_JobRole_Non-Travel_Manager,BusinessTravel_JobRole_Non-Travel_Manufacturing Director,BusinessTravel_JobRole_Non-Travel_Research Director,BusinessTravel_JobRole_Non-Travel_Research Scientist,BusinessTravel_JobRole_Non-Travel_Sales Executive,BusinessTravel_JobRole_Non-Travel_Sales Representative,BusinessTravel_JobRole_Travel_Frequently_Healthcare Representative,BusinessTravel_JobRole_Travel_Frequently_Human Resources,BusinessTravel_JobRole_Travel_Frequently_Laboratory Technician,BusinessTravel_JobRole_Travel_Frequently_Manager,BusinessTravel_JobRole_Travel_Frequently_Manufacturing Director,BusinessTravel_JobRole_Travel_Frequently_Research Director,BusinessTravel_JobRole_Travel_Frequently_Research Scientist,BusinessTravel_JobRole_Travel_Frequently_Sales Executive,BusinessTravel_JobRole_Travel_Frequently_Sales Representative,BusinessTravel_JobRole_Travel_Rarely_Healthcare Representative,BusinessTravel_JobRole_Travel_Rarely_Human Resources,BusinessTravel_JobRole_Travel_Rarely_Laboratory Technician,BusinessTrav

In [14]:
feats = [i for i in data.columns if i not in ['Attrition']]

In [15]:
max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))
for i in feats:
    data[[i]]=data[[i]].apply(max_min_scaler)

In [16]:
data.head()

,Age,Attrition,DistanceFromHome,Education,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,Gender_Female,Gender_Male,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes,BusinessTravel_Department_Non-Travel_Human Resources,BusinessTravel_Department_Non-Travel_Research & Development,BusinessTravel_Department_Non-Travel_Sales,BusinessTravel_Department_Travel_Frequently_Human Resources,BusinessTravel_Department_Travel_Frequently_Research & Development,BusinessTravel_Department_Travel_Frequently_Sales,BusinessTravel_Department_Travel_Rarely_Human Resources,BusinessTravel_Department_Travel_Rarely_Research & Development,BusinessTravel_Department_Travel_Rarely_Sales,BusinessTravel_EducationField_Non-Travel_Human Resources,BusinessTravel_EducationField_Non-Travel_Life Sciences,BusinessTravel_EducationField_Non-Travel_Marketing,BusinessTravel_EducationField_Non-Travel_Medical,BusinessTravel_EducationField_Non-Travel_Other,BusinessTravel_EducationField_Non-Travel_Technical Degree,BusinessTravel_EducationField_Travel_Frequently_Human Resources,BusinessTravel_EducationField_Travel_Frequently_Life Sciences,BusinessTravel_EducationField_Travel_Frequently_Marketing,BusinessTravel_EducationField_Travel_Frequently_Medical,BusinessTravel_EducationField_Travel_Frequently_Other,BusinessTravel_EducationField_Travel_Frequently_Technical Degree,BusinessTravel_EducationField_Travel_Rarely_Human Resources,BusinessTravel_EducationField_Travel_Rarely_Life Sciences,BusinessTravel_EducationField_Travel_Rarely_Marketing,BusinessTravel_EducationField_Travel_Rarely_Medical,BusinessTravel_EducationField_Travel_Rarely_Other,BusinessTravel_EducationField_Travel_Rarely_Technical Degree,BusinessTravel_Gender_Non-Travel_Female,BusinessTravel_Gender_Non-Travel_Male,BusinessTravel_Gender_Travel_Frequently_Female,BusinessTravel_Gender_Travel_Frequently_Male,BusinessTravel_Gender_Travel_Rarely_Female,BusinessTravel_Gender_Travel_Rarely_Male,BusinessTravel_JobRole_Non-Travel_Healthcare Representative,BusinessTravel_JobRole_Non-Travel_Human Resources,BusinessTravel_JobRole_Non-Travel_Laboratory Technician,BusinessTravel_JobRole_Non-Travel_Manager,BusinessTravel_JobRole_Non-Travel_Manufacturing Director,BusinessTravel_JobRole_Non-Travel_Research Director,BusinessTravel_JobRole_Non-Travel_Research Scientist,BusinessTravel_JobRole_Non-Travel_Sales Executive,BusinessTravel_JobRole_Non-Travel_Sales Representative,BusinessTravel_JobRole_Travel_Frequently_Healthcare Representative,BusinessTravel_JobRole_Travel_Frequently_Human Resources,BusinessTravel_JobRole_Travel_Frequently_Laboratory Technician,BusinessTravel_JobRole_Travel_Frequently_Manager,BusinessTravel_JobRole_Travel_Frequently_Manufacturing Director,BusinessTravel_JobRole_Travel_Frequently_Research Director,BusinessTravel_JobRole_Travel_Frequently_Research Scientist,BusinessTravel_JobRole_Travel_Frequently_Sales Executive,BusinessTravel_JobRole_Travel_Frequently_Sales Representative,BusinessTravel_JobRole_Travel_Rarely_Healthcare Representative,BusinessTravel_JobRole_Travel_Rarely_Human Resources,BusinessTravel_JobRole_Travel_Rarely_Laboratory Technician,BusinessTrav

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
train = data[data['Attrition'] !=-1][feats]
train=train.fillna(0)
label = data[data['Attrition'] !=-1]['Attrition']
test = data[data['Attrition'] ==-1][feats]
test = test.fillna(0)
model = LogisticRegression()
model.fit(train,label)
p = model.predict_proba(test)[:,1]
p = (p >= 0.5).astype(int)
test['result']=p
test[['result']].to_csv('/Users/wenjiazhai/Desktop/员工离职预测训练赛/lr_proba.csv',index=False)

/Users/wenjiazhai/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
